In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

In [3]:
filename = '../data/nyc-parking-violations-2020.csv'

df = pd.read_csv(filename,
                 usecols=['Plate ID',  'Registration State',
                        'Vehicle Make', 'Vehicle Color', 'Violation Time', 'Street Name'])

df.head()

,Plate ID,Registration State,Vehicle Make,Violation Time,Street Name,Vehicle Color
0,J58JKX,NJ,HONDA,0523P,43 ST,BK
1,KRE6058,PA,ME/BE,0428P,UNION ST,BLK
2,444326R,NJ,LEXUS,0625A,CLERMONT AVENUE,BLACK
3,F728330,OH,CHEVR,1106A,DIVISION AVE,NaN
4,FMY9090,NY,JEEP,1253A,GRAND ST,GREY


### 🕵️‍♀️ Bellonda's Logic Decoder
**The Business Goal:** Quantify the **Total Addressable Volume** of violations. This establishes our baseline "Universe of Tickets" before we begin filtering or cleaning.

**The Syntax Anatomy:**
* `df.index` → **The Subject** (The vertical spine of the spreadsheet containing all row labels).
* `len(...)` → **The Verb** (Count the total number of items).
* `( )` → **The Scope** (Apply the count strictly to the index labels).

> **💡 The Wisdom Check:**
> While `len(df)` is common, using `len(df.index)` is semantically precise. It explicitly tells the reader: "I am counting the rows (entries), not checking the data structure's depth."

In [4]:
# How many rows are in the data frame when it is read into memory?
len(df.index)

12495734

### 🕵️‍♀️ Bellonda's Logic Decoder
**The Business Goal:** Isolate the **"Ironclad" Data**. We are creating a subset of tickets where *every single field* is populated, representing the "Safe Revenue" (tickets that cannot be dismissed due to missing info).

**The Syntax Anatomy:**
* `df` → **The Source** (Our original raw universe of tickets).
* `.dropna()` → **The Filter** (The "Zero Tolerance" policy).
    * *Implicit Settings:* It defaults to `how='any'` (if 1 value is missing, drop the row) and `axis=0` (drop rows, not columns).
* `all_good_df` → **The Result** (The new DataFrame containing only perfectly complete records).

> **💡 The Wisdom Check:**
> `dropna()` is a blunt instrument(Sledgehammer). It discards a row even if only one minor, non-critical column is missing. Use this when data completeness is non-negotiable (e.g., financial auditing), but be careful not to throw away good data because of a missing specific detail like "Vehicle Color".

In [4]:
# Remove rows with any missing data (i.e., a `NaN` value). How many rows remain after doing this pruning? 
all_good_df = df.dropna()

len(all_good_df.index)

12048375

In [5]:
# How many parking tickets have some missing information, then?
len(df.index) - len(all_good_df.index)

447359

### 🕵️‍♀️ Bellonda's Logic Decoder
**The Business Goal:** Quantify the **Financial Impact of Data Quality**. We are calculating the *Opportunity Cost*—the exact amount of revenue the city is losing specifically because data is missing.

**The Syntax Anatomy:**
* `f'...'` → **The Formatter** (An f-string allows us to inject code logic directly into text).
* `(len(df) - len(all_good))` → **The Delta** (Total Tickets minus Perfect Tickets = The Defective Volume).
* `* 100` → **The Unit Economics** (The value of a single ticket).
* `:,` → **The Humanizer** (A format specifier that automatically adds thousands separators, e.g., turning `1000000` into `1,000,000`).

> **💡 The Wisdom Check:**
> This illustrates the power of Domain Knowledge. Technically, any row with a NaN is "dirty." Business-wise, a NaN equates to a **$100 liability**. Always try to convert your data cleaning metrics into dollar amounts—it gets stakeholders' attention immediately.

In [6]:
# If each parking ticket brings $100 into the city, and missing data means that the ticket can be
# successfully contested, how much money might New York City lose as a result of such missing data?

f'${(len(df.index) - len(all_good_df) ) * 100:,}'  # WOW!

'$44,735,900'

### 🕵️‍♀️ Bellonda's Logic Decoder
**The Business Goal:** Define the **Minimum Viable Record**. We are relaxing our quality control to align with business reality: a ticket is valid as long as the *legally required* fields are present, ignoring non-critical missing info (like "Vehicle Color").

**The Syntax Anatomy:**
* `.dropna(...)` → **The Filter** (Remove rows with missing values).
* `subset=['...']` → **The Target Scope** (Restricts the filter to check *only* these specific columns).
    * If a row has a `NaN` in 'Vehicle Color' but these 4 columns are full, it **survives**.
* `semi_good_df` → **The Recovered Dataset** (A larger set of valid tickets than our first attempt).

> **💡 The Wisdom Check:**
> The `subset` parameter is your most powerful tool for **Nuanced Data Cleaning**. "Clean" is relative. Always ask the Subject Matter Expert: implies "Which specific fields, if missing, actually kill the deal?" Don't clean data just to make it look pretty; clean it to make it *useful*.

In [7]:
# Let's instead assume that a ticket can only be dismissed if the license plate, state, vehicle
# make, and/or street name are all there. Remove rows that are missing one or more of these. 
# How many rows remain?

semi_good_df = df.dropna(subset=['Plate ID', 'Registration State', 'Vehicle Make', 'Street Name'])
len(semi_good_df.index)

12431949

In [8]:
# How many rows did we remove?
len(df.index) - len(semi_good_df.index)

63785

In [9]:
# Assuming $100/ticket, how much money would the city lose as as result of this missing data?

f'${(len(df.index) - len(semi_good_df.index) ) * 100:,}'   # a bit more reasonable...

'$6,378,500'

In [10]:
# Let's instead assume that a ticket can only be dismissed if the license plate, state, and/or
# street name are all there. Remove rows that are missing one or more of these. How many rows remain?
loosest_df = df.dropna(subset=['Plate ID', 'Registration State', 'Street Name'])
len(loosest_df.index)

12494116

In [11]:
# How many rows did we remove?
len(df.index) - len(loosest_df.index)

1618

In [12]:
# Assuming $100/ticket, how much money would the city lose as as result of this missing data?

f'${(len(df.index) - len(loosest_df.index) ) * 100:,}'   # ah, much better...

'$161,800'